**FILL THIS SECTION BY YOUR NAME AND STUDENT CODE** : 

- NAME : Mohammad Mozafari (محمد مظفری)
- STUDENT CODE : 400201167

In [1]:
import math
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
# IMPORT MORE MODULES IF YOU WANT

### Read Dataset

Read the dataset in this section.

In [2]:
df = pd.read_csv('./data/heart.csv')
X = df.loc[:, df.columns != 'target']
y = df['target']
X

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3


### Prepare Dataset

First of all, search for missing values in the dataset. if there are missing values, handle them however you want.

In [3]:
num_missing = df.isnull().sum().sum()
print('Number of missing values:', num_missing)

Number of missing values: 0


Then, read the dataset catalog. There are some categorical features with the "int" type. Encode these features so that you can distinguish between numerical and categorical features.

In [4]:
# My implementation doesn't require this step. All data, categorical or continuous, are treated the same way. In fact if there's any categorical data (with any type other than "int") it must be converted to a numeric value so that we can use it in our algorithm. 

### Declare feature vector and target variable

Here, you are supposed to convert pandas data frame into feature vectors and target variables

In [5]:
X = X.values
y = y.values

### Split data into separate training and test set

Now it's time to split X and y into separate training and test set. You can use the sklearn library for this section.

In [6]:
split_fraction = 0.8
num_total = X.shape[0]
num_train = int(num_total * split_fraction)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

## Implement desicion tree algorithm

In this cell, you are going to implement your decision tree. Feel free to add more arguments to functions or add your desired functions.

In [7]:
class DecisionTree:
     
    def __init__(self, criterion="entropy", max_depth=None, depth=0, min_points=1):
        """
        Parameters:
        
        criterion -- “gini” for the Gini impurity and “entropy” for the information gain. (default “entropy”)
        max_depth -- The maximum depth of the tree. If None, then nodes are expanded until all leaves are pure. (default=None)
        """
        # FILL HERE
        self.max_depth = float('inf') if (max_depth is None) else max_depth
        self.min_points = min_points
        self.left_tree = None
        self.right_tree = None
        self.depth = depth
        self.leaf = False
        self.label = None
        self.split_condition = (None, None)
        self.criterion = criterion

    def compute_gini(self, data):
        result = 1
        total = len(data)
        counter = Counter(data)
        for value in counter.values():
            result -= (value/total) ** 2
        return result

    def compute_entropy(self, data):
        result = 0
        total = len(data)
        counter = Counter(data)
        for value in counter.values():
            p = (value/total)
            result -= p * math.log2(p)
        return result

    def compute_gain(self, root_data, left_data, right_data):
        c1, c2, c3 = 0, 0, 0
        if self.criterion == 'entropy':
            c1 = self.compute_entropy(root_data)
            c2 = self.compute_entropy(left_data)
            c3 = self.compute_entropy(right_data)
        else:
            c1 = self.compute_gini(root_data)
            c2 = self.compute_gini(left_data)
            c3 = self.compute_gini(right_data)
        m = len(left_data)
        n = len(right_data)
        return (c1 - (m/(m+n)) * c2 - (n/(m+n)) * c3)

    def get_avgs(self, numbers):
        numbers = np.sort(numbers)
        return np.convolve(numbers, np.ones(2)) / 2

    def find_best_split_condition(self, X, y):
        data = np.concatenate((X, y.reshape(-1, 1)), axis=1)
        max_gain = 0
        bfeature, bvalue = None, None
        for feature in range(X.shape[1]):
            values = self.get_avgs(X[:, feature])
            for value in values:
                y_left = data[data[:, feature] <= value, -1]
                y_right = data[data[:, feature] > value, -1]
                gain = self.compute_gain(data[:, -1], y_left, y_right)
                if gain > max_gain:
                    max_gain = gain
                    bfeature, bvalue = feature, value
        return bfeature, bvalue
    
    def fit(self, X, y):
        """
        Build a decision tree classifier from the training set (X, y).

        Returns:
        self : Fitted estimator
        """
        data = np.concatenate((X, y.reshape(-1, 1)), axis=1)
        if (X.shape[0] > self.min_points) and (self.depth < self.max_depth):
            bfeature, bvalue = self.find_best_split_condition(X, y)
            if (bfeature is not None) and (bvalue is not None):
                self.split_condition = (bfeature, bvalue)   
                data_left = data[data[:, bfeature] <= bvalue]
                data_right = data[data[:, bfeature] > bvalue]
                self.left = DecisionTree(criterion=self.criterion, max_depth=self.max_depth, depth=self.depth + 1, min_points=self.min_points)
                self.right = DecisionTree(criterion=self.criterion, max_depth=self.max_depth, depth=self.depth + 1, min_points=self.min_points)
                self.left.fit(data_left[:, :-1], data_left[:, -1])
                self.right.fit(data_right[:, :-1], data_right[:, -1])
            else:
                self.leaf = True
                counter = Counter(y)
                self.label = max(counter, key=counter.get)
        else:
            self.leaf = True
            counter = Counter(y)
            self.label = max(counter, key=counter.get)
        return self
    
    def predict_single(self, x):
        if self.leaf:
            return self.label
        if x[self.split_condition[0]] <= self.split_condition[1]:
            return self.left.predict_single(x)
        else:
            return self.right.predict_single(x)
    
    def predict(self, X):
        """
        Predict class value for X.

        Returns:
        y : The predicted classes
        """
        y = np.zeros(X.shape[0])
        for i in range(X.shape[0]):
            pred = self.predict_single(X[i])
            y[i] = pred   
        return y


### Part 1 : Compare Gini and Entropy 

In [8]:
dt_gini = DecisionTree(criterion='gini',)
dt_entropy = DecisionTree(criterion='entropy',)

In this cell, fit both declared trees on the train set and predict values on the test set.

In [9]:
def fit_and_predict(model1, model2, X_train, y_train, X_test):
    model1.fit(X_train, y_train)
    model2.fit(X_train, y_train)
    preds1 = model1.predict(X_test)
    preds2 = model2.predict(X_test)
    return preds1, preds2

Plot confusion matrix for both decision trees.

In [10]:
def compute_accuracy(targets, preds):
    num_data = targets.shape[0]
    accuracy = np.sum(targets == preds) / num_data
    return accuracy

def compute_confusion_matrix(targets, preds):
    TP = np.sum(targets * preds)
    FP = np.sum((1-targets) * preds)
    TN = np.sum((1-targets) * (1-targets))
    FN = np.sum(targets * (1-preds))
    return TP, FP, TN, FN

def report_classification(targets, preds):
    accuracy = compute_accuracy(targets, preds)
    TP, FP, TN, FN = compute_confusion_matrix(targets, preds)
    precision = TP/(TP + FP)
    recall = TP/(TP + FN)
    specificity = TN/(TN + FP)
    f1 = (2 * precision * recall) / (precision + recall)
    return accuracy, precision, recall, specificity, f1


In [11]:
gini_preds, entropy_preds = fit_and_predict(dt_gini, dt_entropy, X_train, y_train, X_train)
gini_report_train = report_classification(y_train, gini_preds)
entropy_report_train = report_classification(y_train, entropy_preds)

gini_preds, entropy_preds = fit_and_predict(dt_gini, dt_entropy, X_train, y_train, X_test)
gini_report_test = report_classification(y_test, gini_preds)
entropy_report_test = report_classification(y_test, entropy_preds)

print('(Accuracy, Precision, Recall, Specificity, F1)')
print('Gini results:')
print('\tOn training data:')
print('\t\t{}'.format(gini_report_train))
print('\tOn test data:')
print('\t\t{}'.format(gini_report_test))

print('Entropy results:')
print('\tOn training data:')
print('\t\t{}'.format(entropy_report_train))
print('\tOn test data:')
print('\t\t{}'.format(entropy_report_test))


(Accuracy, Precision, Recall, Specificity, F1)
Gini results:
	On training data:
		(1.0, 1.0, 1.0, 1.0, 1.0)
	On test data:
		(0.7704918032786885, 0.8108108108108109, 0.8108108108108109, 0.7741935483870968, 0.8108108108108109)
Entropy results:
	On training data:
		(1.0, 1.0, 1.0, 1.0, 1.0)
	On test data:
		(0.7049180327868853, 0.7567567567567568, 0.7567567567567568, 0.7272727272727273, 0.7567567567567567)


### Part 2 : Let's add maximum depth!

Define an array of different maximum depths

In [18]:
from sklearn import tree

max_depths = [2, 5, 7, 10, 15, 20] # FILL THIS LIST WITH DESIRED VALUES
accuracy_scores = []

for max_depth in max_depths:
    dt = DecisionTree(criterion='gini',max_depth=max_depth) # Feel free to change the "entropy" to the "gini"
    # dt = tree.DecisionTreeClassifier(criterion='gini', max_depth=max_depth)
    dt.fit(X_train, y_train)
    test_preds = dt.predict(X_test)
    train_preds = dt.predict(X_train)
    test_report = report_classification(y_test, test_preds)
    train_report = report_classification(y_train, train_preds)
    
    # FIT declared tree to the train set and predict values on the test set. then calcualte accuracy score on the test set
    # Feel free to use the sklearn moudle for calcualting accuracy score.
    accuracy_score = test_report[0]
    accuracy_scores.append(accuracy_score)


In [19]:
for depth, score in zip(max_depths, accuracy_scores):
    print(f"Depth : {depth}, Accuracy : {score}")

Depth : 2, Accuracy : 0.7540983606557377
Depth : 5, Accuracy : 0.8032786885245902
Depth : 7, Accuracy : 0.7704918032786885
Depth : 10, Accuracy : 0.7704918032786885
Depth : 15, Accuracy : 0.7704918032786885
Depth : 20, Accuracy : 0.7704918032786885


Now compare the accuracy score of decision trees with and without using the "max_depth" parameter and discuss the effects of limiting the maximum depth of decision trees.

Answer: When using "max_depth" the accuracy on train data will decrease but this allows more generalization and prevents overfitting. But when we allow infinite depth, although our training accuracy reaches 100% but test accuracy will drop because of overfitting. So it's better to look at validation data and choose the maximum depth where validation accuracy is still increasing.